## Homework

> Note: sometimes your answer might not match one of the options exactly. That's fine. 
Select the option that's closest to your solution.

The goal of this homework is to familiarize you with BentoML and how to build and test an ML production service.

## Background

You are a new recruit at ACME corp. Your manager is emailing you about your first assignment.


## Email from your manager

Good morning recruit! It's good to have you here! I have an assignment for you. I have a data scientist that's built
a credit risk model in a jupyter notebook. I need you to run the notebook and save the model with BentoML and see
how big the model is. If it's greater than a certain size, I'm going to have to request additional resources from 
our infra team. Please let me know how big it is.

Thanks,

Mr McManager


## Question 1

* Install BentoML
* What's the version of BentoML you installed?
* Use `--version` to find out

In [1]:
!bentoml --version

bentoml, version 1.0.7


## Question 2

Run the notebook from module 6 and save the credit risk model with BentoML

How big approximately is the saved BentoML model?

* 924kb
* 724kb
* 114kb
* 8kb

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
import bentoml

In [3]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'

In [4]:
df  = pd.read_csv(data)
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [5]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [9]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [10]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)

xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [14]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    },
)

Model(tag="credit_risk_model:ssuzyxsokktyhchj", path="/Users/patrickwalukagga/bentoml/models/credit_risk_model/ssuzyxsokktyhchj/")

In [15]:
!bentoml models list

 Tag                           Module           Size        Creation Time       
 credit_risk_model:ssuzyxsok…  bentoml.xgboost  197.77 KiB  2022-10-17 22:33:30 
 credit_risk_model:ftxkumcog…  bentoml.xgboost  197.80 KiB  2022-10-17 19:10:10 
 credit_risk_model:tdssyfcod…  bentoml.xgboost  197.77 KiB  2022-10-17 16:14:14 


In [16]:
!bentoml models get credit_risk_model:ssuzyxsokktyhchj

name: credit_risk_model                                                         
version: ssuzyxsokktyhchj                                                       
module: bentoml.xgboost                                                         
labels: {}                                                                      
options:                                                                        
  model_class: Booster                                                          
metadata: {}                                                                    
context:                                                                        
  framework_name: xgboost                                                       
  framework_versions:                                                           
    xgboost: 1.6.2                                                              
  bentoml_version: 1.0.7                                                        
  python_version: 3.9.13    

In [17]:
!bentoml models get credit_risk_model:ssuzyxsokktyhchj --output=json

{
  "name": "credit_risk_model",
  "version": "ssuzyxsokktyhchj",
  "module": "bentoml.xgboost",
  "labels": {},
  "options": {
    "model_class": "Booster"
  },
  "metadata": {},
  "context": {
    "framework_name": "xgboost",
    "framework_versions": {
      "xgboost": "1.6.2"
    },
    "bentoml_version": "1.0.7",
    "python_version": "3.9.13"
  },
  "signatures": {
    "predict": {
      "batchable": false
    }
  },
  "api_version": "v2",
  "creation_time": "2022-10-17T19:33:30.679667+00:00"
}


## Another email from your manager

Great job recruit! Looks like I won't be having to go back to the procurement team. Thanks for the information.

However, I just got word from one of the teams that's using one of our ML services and they're saying our service is "broken"
and their trying to blame our model. I looked at the data their sending and it's completely bogus. I don't want them
to send bad data to us and blame us for our models. Could you write a pydantic schema for the data that they should be sending?
That way next time it will tell them it's their data that's bad and not our model.

Thanks,

Mr McManager

## Question 3

Say you have the following data that you're sending to your service:

```json
{
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}
```

In [32]:
!cat service.py

import numpy as np

import bentoml
from bentoml.io import JSON
from pydantic import BaseModel


class CreditApplication(BaseModel):
    """Pydantic schema for the data being sent

    {
        "name": "Tim",
        "age": 37,
        "country": "US",
        "rating": 3.14
    }
    """
    name: str
    age: int
    country: str
    rating: float


## Email from your CEO

Good morning! I hear you're the one to go to if I need something done well! We've got a new model that a big client
needs deployed ASAP. I need you to build a service with it and test it against the old model and make sure that it performs
better, otherwise we're going to lose this client. All our hopes are with you!

Thanks,

CEO of Acme Corp

## Question 4

We've prepared a model for you that you can import using:

```bash
curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel
bentoml models import coolmodel.bentomodel
```

What version of scikit-learn was this model trained with?

* 1.1.1
* 1.1.2
* 1.1.3
* 1.1.4
* 1.1.5

In [18]:
!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1724  100  1724    0     0    940      0  0:00:01  0:00:01 --:--:--   940


In [19]:
!bentoml models import coolmodel.bentomodel

Model(tag="mlzoomcamp_homework:qtzdz3slg6mwwdu5") imported


In [20]:
!bentoml models get mlzoomcamp_homework:qtzdz3slg6mwwdu5 --output=json

{
  "name": "mlzoomcamp_homework",
  "version": "qtzdz3slg6mwwdu5",
  "module": "bentoml.sklearn",
  "labels": {},
  "options": {},
  "metadata": {},
  "context": {
    "framework_name": "sklearn",
    "framework_versions": {
      "scikit-learn": "1.1.1"
    },
    "bentoml_version": "1.0.7",
    "python_version": "3.9.12"
  },
  "signatures": {
    "predict": {
      "batchable": false
    }
  },
  "api_version": "v1",
  "creation_time": "2022-10-13T20:42:14.411084+00:00"
}


## Question 5 

Create a bento out of this scikit-learn model. This will require installing scikit-learn like this:
```bash
pip install scikit-learn
```

Hint: The output type for this endpoint should be NumpyNdarray()

Send this array to the bento:

```
[[6.4,3.5,4.5,1.2]]
```

You can use curl or the Swagger UI. What value does it return? 

* 0
* 1
* 2
* 3

#### Run command below to serve the homework_service
```
bentoml serve homework_service:svc --reload
```

In [27]:
import requests

result = requests.post(
   "http://127.0.0.1:3000/classify",
   headers={"content-type": "application/json"},
   data="[[6.4, 3.5, 4.5, 1.2]]",
).json()
result

[1]

In [28]:
!curl -X POST \
   -H "content-type: application/json" \
   --data "[[6.4, 3.5, 4.5, 1.2]]" \
   http://127.0.0.1:3000/classify

[1]

In [26]:
import bentoml

model_ref = bentoml.sklearn.get("mlzoomcamp_homework:qtzdz3slg6mwwdu5")
model_runner = model_ref.to_runner()
model_runner.init_local()
model_runner.predict.run([[6.4, 3.5, 4.5, 1.2]])

'Runner.init_local' is for debugging and testing only.
/Users/patrickwalukagga/.local/share/virtualenvs/mlzoomcamp-twe4lI-M/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([1])

## Question 6

Ensure to serve your bento with `--production` for this question

Install locust using:

```bash
pip install locust
```

Use the following locust file: [locustfile.py](locustfile.py)

Ensure that it is pointed at your bento's endpoint (In case you didn't name your endpoint "classify")

<img src="resources/classify-endpoint.png">

Configure 100 users with ramp time of 10 users per second. Click "Start Swarming" and ensure that it is working

Now download a second model with this command:

```bash
curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel
```

Or you can download with this link as well:
[https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel](https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel)

Now import the model:

```bash
bentoml models import coolmodel2.bentomodel
```

Update your bento's runner tag and test with both models. Which model allows more traffic (more throughput) as you ramp up the traffic?
Remember to turn off and turn on your bento service between changing the model tag. Use Ctl-C to close the service. Then call bentoml serve

Test out the first model and the second model, which one performance better at higher volumes?

* The first model
* The second model

In [29]:
!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1728  100  1728    0     0   1097      0  0:00:01  0:00:01 --:--:--  1096


In [30]:
!bentoml models import coolmodel2.bentomodel

Model(tag="mlzoomcamp_homework:jsi67fslz6txydu5") imported


In [31]:
!bentoml models get mlzoomcamp_homework:jsi67fslz6txydu5 --output=json

{
  "name": "mlzoomcamp_homework",
  "version": "jsi67fslz6txydu5",
  "module": "bentoml.sklearn",
  "labels": {},
  "options": {},
  "metadata": {},
  "context": {
    "framework_name": "sklearn",
    "framework_versions": {
      "scikit-learn": "1.1.1"
    },
    "bentoml_version": "1.0.7",
    "python_version": "3.9.12"
  },
  "signatures": {
    "predict": {
      "batchable": true,
      "batch_dim": [
        0,
        0
      ]
    }
  },
  "api_version": "v1",
  "creation_time": "2022-10-14T14:48:43.330446+00:00"
}


- **Second Model** gives better performance at higher volumes

## Email from marketing

Hello ML person! I hope this email finds you well. I've heard there's this cool new ML model called Stable Diffusion.
I hear if you give it a description of a picture it will generate an image. We need a new company logo and I want it
to be fierce but also cool, think you could help out?

Thanks,

Mike Marketer


## Question 7 (optional)

Go to this Bento deployment of Stable Diffusion: http://54.176.205.174/ (or deploy it yourself)

Use the txt2image endpoint and update the prompt to: "A cartoon dragon with sunglasses". 
Don't change the seed, it should be 0 by default

What is the resulting image?

### #1
<img src="resources/dragon1.jpeg">

### #2 
<img src="resources/dragon2.jpeg">

### #3 
<img src="resources/dragon3.jpeg">

### #4
<img src="resources/dragon4.jpeg">

- **Image #3**

## Submit the results

* Submit your results here: https://forms.gle/Hh9FWy6LGXk3wJYs8
* You can submit your solution multiple times. In this case, only the last submission will be used 
* If your answer doesn't match options exactly, select the closest one